# Fine-tune Whisper Audio Transcribe Model





### Insights and Code References Attribution

This work incorporates insights and code snippets courtesy of Hugging Face, published on Hugging Face's blog. We extend our gratitude to the original authors and Hugging Face for sharing their valuable resources with the community.

Here are few resources used for reference.

https://huggingface.co/blog/fine-tune-whisper

https://github.com/Vaibhavs10/fast-whisper-finetuning/blob/main/Whisper_w_PEFT.ipynb

https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0/viewer/pa-IN/train?f[sentence][min]=16&f[sentence][max]=22&f[sentence][transform]=length

Environment
We will run this code on google colab with T4/A4 GPU

## Install required Libraries


transformers - Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. 

accelerate - Accelerate  enables the same  code to be run across different distributed configurations.

peft -  Parameter Efficient Fine Tuning 

datasets - for  accessing and sharing data

bitsandbytes - For quantization 



### What is Whisper

Whisper is an audio processing model developed by OpenAI, designed for robust speech recognition and capable of transcribing and translating spoken language from audio files. It leverages deep learning techniques to understand and process audio inputs with high accuracy, even in challenging environments with background noise or different accents. Whisper aims to bridge communication gaps by providing accessible and efficient audio-to-text conversion.

## Understanding of Basic Concepts




### peft - Parameter Efficient Fine Tuning

Parameter Efficient Fine-Tuning (PEFT) refers to a set of techniques in machine learning aimed at updating large pre-trained models in a way that requires altering only a small fraction of the model's parameters. This approach is particularly useful when working with very large models, such as those used in natural language processing (NLP) and computer vision, where full model retraining can be prohibitively expensive in terms of computational resources and time.


### LoRA

In the context of parameter-efficient fine-tuning for machine learning models, LoRA and QLoRa refer to specific techniques designed to update large pre-trained models more efficiently.

LoRA (Low-Rank Adaptation):

Low-Rank Adaptation is a technique used to fine-tune large pre-trained models in a parameter-efficient manner. Instead of updating all the parameters of a model, LoRA focuses on adapting only a small set of parameters, typically by introducing and training low-rank matrices that modify the weights of specific layers (like the attention or feed-forward layers in a Transformer model). This approach allows for significant customization of the model to new tasks with minimal changes to the overall parameter set, preserving the original strengths of the pre-trained model while adapting it to specific needs.

Imagine you have a huge, complex book (representing a large pre-trained model) that you want to customize slightly without rewriting the whole thing. LoRA allows you to do just that by adding a few notes or annotations (small changes) to some pages, making the book more relevant to your specific needs without altering its original content significantly. In technical terms, LoRA introduces small, trainable adjustments to certain parts of the model while keeping the vast majority of the original parameters fixed. This makes the fine-tuning process much more efficient, as you're only tweaking a tiny fraction of the model.

![lora](lora.jpeg)

The image above is designed to visually explain the concept of Low-Rank Adaptation (LoRA) within Parameter Efficient Fine-Tuning (PEFT), with a focus on resembling a neural network. It illustrates how specific LoRA modules act as targeted enhancements to the neural network, allowing for precise and efficient adjustments to adapt the model for new tasks.


### QLoRa (Quantized Low-Rank Adaptation):

Quantization, in the context of machine learning, involves reducing the precision of the numerical representations used in a model. For example, reducing floating-point numbers from 32-bit to 8-bit integers. This process helps decrease the model's memory footprint and can speed up inference, making the model more efficient, especially on hardware with limited computational resources or specific accelerators designed to work with lower-precision arithmetic.

Building on the previous analogy, if LoRA is like adding handwritten notes to the book, QLoRa is akin to using a shorthand or a more condensed notation for those notes. QLoRa further compresses the changes you make (the annotations or notes) by quantizing them, which means representing these adjustments with fewer bits. This not only makes the fine-tuning process more efficient but also reduces the memory and computational resources needed to apply these customizations.

![qlora](qlora.jpeg)

In very simple terms, both LoRA and QLoRa are about making smart, minimal updates to a large, pre-existing "knowledge base" (the pre-trained model) to adapt it for specific tasks. LoRA does this by adding small modifications, and QLoRa goes a step further by compressing these modifications to be even more resource-efficient.

## Install Libraries



In [ ]:
#!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes accelerate

!pip install -q git+https://github.com/huggingface/peft.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 k

In [ ]:
!pip install --upgrade  transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
!pip install --upgrade peft

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Login to huggingface

Enter your Access Tokens when prompted. 

We can find the Access Tokens from settings page [here](https://huggingface.co/settings/tokens):

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Define Whisper model name and task details.

In [ ]:
model_name_or_path = "openai/whisper-small"
task = "transcribe"

We will fine tune on the punjabi language data

In [ ]:
dataset_name = "mozilla-foundation/common_voice_13_0"
language = "Punjabi"
language_abbr = "pa-IN" # Short hand code for the language we want to fine-tune

# Load Dataset

Accept the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_13_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0). After that we can download the data .



In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test", use_auth_token=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 712it [00:00, 102796.61it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 285it [00:00, 97153.50it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1414it [00:00, 106814.10it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 77it [00:00, 73250.49it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 997
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 459
    })
})


Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 997
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 459
    })
})


## Prepare Feature Extractor, Tokenizer and Data

The automatic speech recognition pipeline can be de-composed into three stages:
1. A feature extractor which pre-processes the raw audio-inputs
2. The model which performs the sequence-to-sequence mapping
3. A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


To simplify using the feature extractor and tokenizer, we can _wrap_ both into a single `WhisperProcessor` class. This processor object can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/0ae81662e85c06e585d3ad85aecd7c518d5e2785aed4c0b5154156b935ad582c/pa-IN_train_0/common_voice_pa-IN_23337849.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 48000}, 'sentence': 'ਕਲੱਬਾਂ ਦੇ ਬੈਨਰ ਲੱਗੇ ਹੋਏ ਸਨ'}


Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/0ae81662e85c06e585d3ad85aecd7c518d5e2785aed4c0b5154156b935ad582c/pa-IN_train_0/common_voice_pa-IN_23337849.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        1.57673874e-12, -5.00932629e-13, -3.45390383e-13]), 'sampling_rate': 16000}, 'sentence': 'ਕਲੱਬਾਂ ਦੇ ਬੈਨਰ ਲੱਗੇ ਹੋਏ ਸਨ'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

Make yourself some tea 🍵, depending on dataset size, this might take 20-30 minutes ⏰

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/997 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/459 [00:00<?, ? examples/s]

In [ ]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 997
})

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric.

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will create a gradio interface to transcribe speech in Punjabi.

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper checkpoint. Again, this
is trivial through use of 🤗 Transformers!

To reduce our models memory footprint, we load the model in 8bit, this means we quantize the model to use 1/4th precision (when comapared to float32) with minimal loss to performance. To read more about how this works, head over [here](https://huggingface.co/blog/hf-bitsandbytes-integration).

In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.43.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [ ]:
!pip freeze | grep bitsandbytes

bitsandbytes==0.43.0


In [ ]:
#!pip install bitsandbytes
!pip install --upgrade bitsandbytes

In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.43.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [ ]:
pip freeze | grep bitsandbytes

bitsandbytes==0.43.0


In [ ]:
from transformers import WhisperForConditionalGeneration

#model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

from transformers import  BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.float16,
)

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, quantization_config=bnb_config, device_map="auto")


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
!pip install --upgrade peft

In [ ]:
#!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio
!pip install --upgrade  transformers accelerate

### Post-processing on the model

Finally, we need to apply some post-processing steps on the 8-bit model to enable training. We do so by first freezing all the model layers, and then cast the layer-norm and the output layer in `float32` for training and model stability.

In [ ]:
#from peft import prepare_model_for_int8_training
from peft import prepare_model_for_kbit_training

#model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")
model = prepare_model_for_kbit_training(model)

Since the Whisper model uses Convolutional layers in the Encoder, checkpointing disables grad computation to avoid this we specifically need to make the inputs trainable.

In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

### Apply Low-rank adapters (LoRA) to the model

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.442854145857274


We are ONLY using **1%** of the total trainable parameters, thereby performing **Parameter-Efficient Fine-Tuning**

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="aibotteachesai/test",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=10,
    num_train_epochs=1,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=50,
    max_steps=200, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)




Fine-tuning a model with PEFT comes with a few caveats.

1. We need to explicitly set `remove_unused_columns=False` and `label_names=["labels"]` as the PeftModel's forward doesn't inherit the signature of the base model's forward.

2. Since INT8 training requires autocasting, we cannot use the native `predict_with_generate` call in Trainer as it doesn't automatically cast.

3. Similarly, since we cannot autocast, we cannot pass the `compute_metrics` to `Seq2SeqTrainer` so we'll comment it out whilst instantiating the Trainer.

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
50,1.436300,0.766590
100,0.443700,0.402472
150,0.249300,0.347409
200,0.189400,0.317255


TrainOutput(global_step=200, training_loss=0.5796979951858521, metrics={'train_runtime': 642.9819, 'train_samples_per_second': 2.488, 'train_steps_per_second': 0.311, 'total_flos': 4.6900932304896e+17, 'train_loss': 0.5796979951858521, 'epoch': 1.6})

Now that our model is fine-tuned, we can push the model on to Hugging Face Hub, this will later help us directly infer the model from the model repo.

In [ ]:
peft_model_id = "aibotteachesai/whisper-small-punjabi-1-200steps"
model.push_to_hub(peft_model_id)

# Evaluation and Inference

we've successfully fine-tuned our model. Now let's run inference on it.


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_config = PeftConfig.from_pretrained(peft_model_id)


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)


model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto"
)


model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

adapter_config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [ ]:
#!pip install --upgrade gradio

In [ ]:
from transformers import pipeline
import gradio as gr


pipe = pipeline(model=peft_model_id)

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Punjabi",
    description="Realtime demo for Punjabi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ff6b7602ef305c6641.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Sample screenshots of the fine tuned whisper transcribe model output

![punjabi_transcribe_screenshot_1](punjabi_transcribe_screenshot_1.png)

![punjabi_transcribe_screenshot_2](punjabi_transcribe_screenshot_2.png)